In [4]:
import os
import textwrap
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain import PromptTemplate

In [5]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()

## Elegir si enseñar o recordar (evita el uso de tokens en OpenAI)

### 1. Enseñar

In [6]:
# Carga de documentos y extracción de información
# (asegúrate de que haya PDFs en la carpeta documentos)
documents = DirectoryLoader('./documentos/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
# Tratameinto de caracteres indeseados
for d in documents:
    d.page_content = d.page_content.replace('\n', ' ').replace('\t', ' ')
# Separador de texto (límita el texto para que sea soportado por el LLM)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)
textos = text_splitter.split_documents(documents)
# Se instruye (creación de un knowledge-base) con la info
# proporcionada
persist_directory = input('¿Cómo se llama este knoledge-base?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=textos, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)
# Se guarda el conocimiento generado en el directorio actual
# para ser usado sin tener que instruir al LLM
vectordb.persist()
vectordb = None
os.system(f'zip -r db.zip ./{persist_directory}')

c:\Users\luisr\OneDrive\Desktop\Consultoría\projects\prometeo\chatbots\prometeo\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


1

### 2. Recordar (si y solo si Enseñar=Done)

In [7]:
# Now we can load the persisted database from disk, and use it as normal. 
persist_directory = input('¿Qué knoledge-base deseas usar?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
#retriever = vectordb.as_retriever()

In [8]:
# Se especifica el LLM de OpenAI
turbo_llm = ChatOpenAI(
    temperature=0.7,
    model_name='gpt-3.5-turbo-0125'
)
# Se personaliza el LLM
template = """
Eres Prometeo, un asistente personal de lectura que habla Español.

Tu tarea consiste en:

1. Ser carismático y ofrecer información sobre ti y tus funciones.

2. Leer detalladamente la información proporcionada en documentos
de texto en formato PDF, para ser capaz de dar respuestas a cualquier tipo de pregunta,
en especial a preguntas puntuales.

SIEMPRE debes responder con base al contexto proporcionado aqui: {context}.
        
Si no sabes la respuesta, puedes decir: 'No sé'.

NUNCA hables del contexto.

Teniendo lo anterior en cuenta, responde la siguiente pregunta: {question}
"""

prometeo_prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)
# Creación de cadena
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs={"prompt": prometeo_prompt},
                                       return_source_documents=True)
# Formato y referencias de las respuestas
def wrap_text_preserve_newlines(text, width=70):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\nReferencias:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

c:\Users\luisr\OneDrive\Desktop\Consultoría\projects\prometeo\chatbots\prometeo\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = qa_chain(query)
process_llm_response(llm_response)

¿Cuáles son todas las hipótesis planteadas?


c:\Users\luisr\OneDrive\Desktop\Consultoría\projects\prometeo\chatbots\prometeo\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Las hipótesis planteadas son:

1. Descubrir las relaciones existentes entre el proceso de toma de
decisiones estratégicas, la calidad de las decisiones y la eficacia
organizativa en el conjunto de organizaciones estudiadas.
2. Plantear un conjunto de hipótesis con consistencia teórica y
empírica para ser verificadas en futuras investigaciones.
3. Proporcionar evidencia empírica en un campo disciplinario de
exigencia creciente para el sector público.
4. Sugerir buenas prácticas para mejorar las posibilidades de éxito en
la dirección de las organizaciones.

Referencias:
documentos\19n35a04.pdf
documentos\19n35a04.pdf
documentos\19n35a04.pdf
documentos\19n35a04.pdf
documentos\19n35a04.pdf


In [12]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

{'query': '¿Cuáles son todas las hipótesis planteadas?',
 'result': 'Las hipótesis planteadas son las siguientes:\n\n1. Descubrir las relaciones existentes entre proceso de toma de decisiones estratégicas, calidad de las decisiones y eficacia organizativa en el conjunto de organizaciones estudiadas.\n2. Plantear un conjunto de hipótesis con consistencia teórica y empírica para ser verificadas en futuras investigaciones.\n3. Proporcionar evidencia empírica en un campo disciplinario de exigencia creciente para el sector público.\n4. Sugerir buenas prácticas para mejorar las posibilidades de éxito en la dirección de las organizaciones.',
 'source_documents': [Document(page_content='y medidas e índices de fiabilidad  se presentan a continuación: Racionalidad /g37/g81/g80/g384/g75/g69/g86/g81  cognitivo Flexibilidad  cognitiva  /g37/g81/g80/g384/g75/g69/g86/g81  afectivo  Calidad de  la decisión  /g39/g383/g69/g67/g69/g75/g67/g2 organizativa Politización  REV innovar 19(35).indb   39 08/04/

In [13]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = qa_chain(query)
process_llm_response(llm_response)

¿Cuáles son los objetivos de la investigación?
Los objetivos de la investigación son:
1. Descubrir las relaciones existentes entre el proceso de toma de
decisiones estratégicas, la calidad de las decisiones y la eficacia
organizativa en las organizaciones estudiadas.
2. Plantear un conjunto de hipótesis con consistencia teórica y
empírica para ser verificadas en futuras investigaciones.
3. Proporcionar evidencia empírica en un campo disciplinario de
exigencia creciente para el sector público.
4. Sugerir buenas prácticas para mejorar las posibilidades de éxito en
la dirección de las organizaciones.

Referencias:
documentos\19n35a04.pdf
documentos\19n35a04.pdf
documentos\19n35a04.pdf
documentos\19n35a04.pdf
documentos\19n35a04.pdf


In [14]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

{'query': '¿Cuáles son los objetivos de la investigación?',
 'result': 'Los objetivos de la investigación son:\n1. Descubrir las relaciones existentes entre el proceso de toma de decisiones estratégicas, calidad de las decisiones y eficacia organizativa en el conjunto de organizaciones estudiadas.\n2. Plantear un conjunto de hipótesis con consistencia teórica y empírica para ser verificadas en futuras investigaciones.\n3. Proporcionar evidencia empírica en un campo disciplinario de exigencia creciente para el sector público.\n4. Sugerir buenas prácticas para mejorar las posibilidades de éxito en la dirección de las organizaciones.',
 'source_documents': [Document(page_content='40 ADM. PÚBLICA INNOVAR REV. INNOVAR. Vol. 19, No. 35, SEPTIEMBRE –DICIEMBRE DE 2009  Métodos  El primer objetivo de la investigación consiste en des- cubrir en qué magnitud la calidad de las decisiones es- tratégicas impacta sobre la eficacia organizativa en las  organizaciones públicas analizadas. Para este efe

In [15]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = qa_chain(query)
process_llm_response(llm_response)

¿Cuáles son los hallazgos de la investigación?
Los hallazgos de la investigación son que la calidad de las decisiones
estratégicas influye en la eficacia organizativa en las organizaciones
públicas estudiadas, que la racionalidad del proceso de toma de
decisiones estratégicas impacta positivamente en la calidad de las
decisiones, que la flexibilidad cognitiva también impacta
positivamente en la calidad de las decisiones estratégicas, y que el
conflicto de ideas contribuye a lograr mayores niveles de calidad en
las decisiones estratégicas.

Referencias:
documentos\19n35a04.pdf
documentos\19n35a04.pdf
documentos\19n35a04.pdf
documentos\19n35a04.pdf
documentos\19n35a04.pdf


In [ ]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

In [16]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = qa_chain(query)
process_llm_response(llm_response)

¿Cuáles son las sugerencias para lograr mayores niveles de éxito en la toma de decisiones estratégicas?
Las sugerencias para lograr mayores niveles de éxito en la toma de
decisiones estratégicas incluyen favorecer la racionalidad en las
decisiones estratégicas, buscar información de forma profunda y
exhaustiva, analizar la información detalladamente, generar
alternativas exhaustivas y realizar una selección rigurosa y analítica
de la opción estratégica. También se recomienda favorecer la
flexibilidad cognitiva estimulando la discusión de muchas ideas
diferentes, el análisis de múltiples perspectivas, las diferencias de
opinión entre los participantes y la incorporación de nueva
información de forma recurrente. Además, se sugiere favorecer el
conflicto cognitivo incentivando la discusión de nuevas ideas, la
aceptación de ideas originales, la participación amplia y creativa de
los directivos, y la reconsideración de las posiciones alcanzadas
inicialmente.

Referencias:
documentos\19n35a0

In [ ]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

In [11]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)


Eres Prometeo, un asistente personal de lectura que habla Español.

Tu tarea consiste en:

1. Ser carismático y ofrecer información sobre ti y tus funciones.

2. Leer detalladamente la información proporcionada en documentos
de texto en formato PDF, para ser capaz de dar respuestas a cualquier tipo de pregunta,
en especial a preguntas puntuales.

SIEMPRE debes responder con base al contexto proporcionado aqui: {context}.
        
Si no sabes la respuesta, puedes decir: 'No sé'.

NUNCA hables del contexto.

Teniendo lo anterior en cuenta, responde la siguiente pregunta: {question}

